La voiture : étude d'un fait social

Par Alexandre Pakin et Gilles Massoulié


Introduction

L’usage de la voiture constitue un objet central d’analyse en sociologie, tant il structure les mobilités quotidiennes, les trajectoires sociales et les rapports aux territoires en France. Loin d’être un simple outil de déplacement, l’automobile peut être appréhendée comme un fait social, révélateur d’inégalités de classe, de genre, d’âge et de territoire.

Cette étude a été motivée par une [Vidéo de vulgarisation sociologique sur l’automobile](https://www.youtube.com/watch?v=tR6aujaUh_g)
 proposant une synthèse critique de plusieurs travaux récents sur l’automobile et ses usages sociaux. En mettant en évidence la dimension profondément inégalitaire de la conduite et de la possession d’un véhicule, cette vidéo invite à revenir aux données empiriques afin de reproduire et discuter certaines conclusions à partir de sources ouvertes. Le présent projet s’inscrit ainsi dans une démarche de réanalyse quantitative, visant à confronter des hypothèses sociologiques établies aux données disponibles.

La littérature récente permet de dégager plusieurs axes majeurs. D’une part, des travaux généraux de sociologie de l’automobile, comme [Sociologie de l’automobile](https://www.cairn.info/sociologie-de-l-automobile--9782707196433.htm) de Jean-Pierre Orfeuil et Frédéric Héran (2022) ou [Idées reçues sur l’automobile](https://www.cairn.info/idees-recues-sur-l-automobile--9782715405666.htm) de Frédéric Héran (2021), proposent un cadre théorique permettant de déconstruire les représentations dominantes associées à la voiture. D’autre part, plusieurs études mettent en évidence des usages différenciés selon les caractéristiques sociales des individus. Les dimensions d’âge et de genre sont notamment abordées dans des travaux portant sur la jeunesse automobile ou sur les inégalités et comportements genrés au volant, tandis que d’autres recherches analysent l’automobile comme un marqueur de position sociale et un révélateur des inégalités de classe, notamment à travers les politiques de mobilité et de sécurité routière.

Dans le prolongement de ces travaux, ce projet vise à analyser empiriquement l’usage de la voiture en France à partir de données issues de trois API publiques (data.gouv.fr, INSEE et transport.data.gouv.fr). L’objectif est d’identifier des régularités statistiques et des inégalités sociales liées à l’automobile, tout en illustrant l’apport des outils de programmation en Python pour l’analyse quantitative de phénomènes sociaux.